In [5]:
from langchain.llms.openai import OpenAI #llm 모델 선택 (davinci 는 비싸)
from langchain.chat_models import ChatOpenAI #chat 모델 선택, #ChatAnthropic

#llm = OpenAI()
chat = ChatOpenAI() # ChatAnthropic() 모델 바로 바꿀수 있어
# chat모델은 chat에 특화
# a = llm.predict("How many planets are there")
b = chat.predict("How many planets are there") 
b

'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [6]:
from langchain.chat_models import ChatOpenAI #chat 모델 선택, #ChatAnthropic
from langchain.prompts import PromptTemplate, ChatPromptTemplate #ChatPromptTemplate : 은 아래 'messages'로 부터 프롬프트를 만들고 / PromptTemplate : 은 string으로부터 만듬

chat = ChatOpenAI(temperature=0.1) 

template = PromptTemplate.from_template("What is the distance between {place_a} and {place_b}? Also, what is your name?")
prompt = template.format(place_a='Mexico', place_b='Thailand')

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles). My name is Assistant.'

In [7]:
# # predict message
# from langchain.schema import HumanMessage, AIMessage, SystemMessage #system message : 우리가 llm설정들을 제공하기 위한 message
# messages = [
#     SystemMessage(content = "You are a geography expert. And you only reply in {language}"),
#     AIMessage(content = "안녕하세요, 저는 대한민국 대동여지도를 만든 {name} 입니다."),
#     HumanMessage(content = "What is the distance between {place_a} and {place_b}? Also, what is your name?")
# ]

# chat.predict_messages(messages)

In [8]:
# tempplate 생성
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}"),
    ("ai", "안녕하세요, 저는 대한민국 대동여지도를 만든 {name} 입니다."),
    ("human", "What is the distance between {place_a} and {place_b}? Also, what is your name?")
])

prompt = template.format_messages(
    language = "Greek",
    name = "Socrates",
    place_a = "Mexico",
    place_b = "Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [9]:
# Output Parser필요한 이유는 llm의 응답을 변형해야할 때가 있기 때문 (리스트로 변환하던지)
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse("Hello,how, are,you")

['Hello', 'how', 'are', 'you']

In [16]:
template = ChatPromptTemplate.from_messages(
    [("system","You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",),
     ("human","{question}"),]
)
prompt = template.format_messages(max_items=10, question="What are the colors?")
result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['Red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [17]:
# Better way.
# 코드를 다시 정리 해보자 (여기서 부터 코드 다시 돌려봐)
chat = ChatOpenAI(temperature=0.1) 

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

template = ChatPromptTemplate.from_messages(
    [("system","You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",),
     ("human","{question}"),]
)

# Chain을 만들거야
chain = template | chat | CommaOutputParser()
chain.invoke({  #chain 실행, dictionary형태로
    "max_items" : 5,
    "question" : "What are the Pokemons"}) 

['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle', 'Jigglypuff']